Introducción a Bases de Datos MySQL con Python
===

Usando las técnicas a continuación, podrá integrar de manera eficiente una base de datos MySQL con una aplicación de Python. Tendrá la posibilidad de construir una pequeña base de datos MySQL para un sistema de clasificación de películas y aprenderá a conectar, consultar y modificar sus datos.

El primer paso es realizar la instalación MySQL en su entorno local:
- [Installing and Upgrading MySQL](https://dev.mysql.com/doc/refman/5.7/en/installing.html)
- [MySQL Community Downloads](https://dev.mysql.com/downloads/installer/)

Como prerequisito final, debe instalar el [conector de MySQL para Python](https://github.com/mysql/mysql-connector-python):
- Ejecute el comando ```pip install mysql-connector-python```en consola (bash, cmd, etc.)
- Pruebe la instalación, ingrese por consola a ```python3``` y ejecute ```import mysql.connector```

## Contenido
>- [Estableciendo una conexión](#Estableciendo-una-conexión)
>- [Creación, modificación y eliminación de una tabla](#Creación,-modificación-y-eliminación-de-una-tabla)
>- [Insertando registros en una tabla](#Insertando-registros-en-una-tabla)
>- [Consultando registros de la base de datos](#Consultando-registros-de-la-base-de-datos)
>- [Filtrando resultados usando ```where```](#Filtrando-resultados-usando-where)
>- [Manejando múltiples tablas usando ```join```](#Manejando-múltiples-tablas-usando-join)
>- [Actualización y eliminación de registros](#Actualización-y-eliminación-de-registros)
>- [Otras formas de conectarse con MySQL](#Otras-formas-de-conectarse-con-MySQL)

In [ ]:
from getpass import getpass
from mysql.connector import connect, Error

## Estableciendo una conexión

In [ ]:
# Establecemos una conexión a la base de datos MySQL

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
    ) as connection:
        print(connection)
except Error as e:
    print(e)

In [ ]:
# Eliminamos la base de datos en MySQL si esta existe

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
    ) as connection:
        create_db_query = "DROP DATABASE IF EXISTS online_movie_rating"
        with connection.cursor() as cursor:
            cursor.execute(create_db_query)
except Error as e:
    print(e)

In [ ]:
# Creamos una nueva base de datos en MySQL

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
    ) as connection:
        create_db_query = "CREATE DATABASE online_movie_rating"
        with connection.cursor() as cursor:
            cursor.execute(create_db_query)
except Error as e:
    print(e)

In [ ]:
# Verificamos que la base de datos haya sido creada

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
    ) as connection:
        show_db_query = "SHOW DATABASES"
        with connection.cursor() as cursor:
            cursor.execute(show_db_query)
            print('online_movie_rating', 
                  'exists :)' if 'online_movie_rating' in [db[0] for db in cursor] \
                  else 'not exists :(')
except Error as e:
    print(e)

In [ ]:
#  Creamos una conexión a la nueva base de datos

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database="online_movie_rating",
    ) as connection:
        print(connection)
except Error as e:
    print(e)

## Creación, modificación y eliminación de una tabla

![schema-movies.webp](./images/schema-movies.webp)

In [ ]:
# Creamos una variable con la conexión

try:
    connection = connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database="online_movie_rating",
    )
    print(connection)
except Error as e:
    print(e)

In [ ]:
# Creamos la tabla "movies"

create_movies_table_query = """
    CREATE TABLE movies(
        id INT AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(100),
        release_year YEAR(4),
        genre VARCHAR(100),
        collection_in_mil INT
    )
"""
with connection.cursor() as cursor:
    cursor.execute(create_movies_table_query)
    connection.commit()

In [ ]:
# Creamos la tabla reviewers

create_reviewers_table_query = """
    CREATE TABLE reviewers (
        id INT AUTO_INCREMENT PRIMARY KEY,
        first_name VARCHAR(100),
        last_name VARCHAR(100)
    )
"""
with connection.cursor() as cursor:
    cursor.execute(create_reviewers_table_query)
    connection.commit()

In [ ]:
# Creamos la tabla ratings

create_ratings_table_query = """
    CREATE TABLE ratings (
        movie_id INT,
        reviewer_id INT,
        rating DECIMAL(2,1),
        FOREIGN KEY(movie_id) REFERENCES movies(id),
        FOREIGN KEY(reviewer_id) REFERENCES reviewers(id),
        PRIMARY KEY(movie_id, reviewer_id)
    )
"""
with connection.cursor() as cursor:
    cursor.execute(create_ratings_table_query)
    connection.commit()

In [ ]:
# Creamos todas las tablas en secuencia

with connection.cursor() as cursor:
    try:
        cursor.execute(create_movies_table_query)
        cursor.execute(create_reviewers_table_query)
        cursor.execute(create_ratings_table_query)
        connection.commit()
    except:
        connection.rollback()

In [ ]:
# Vemos estructura de tabla con DESCRIBE

show_table_query = "DESCRIBE movies"
with connection.cursor() as cursor:
    cursor.execute(show_table_query)
    # Fetch rows from last executed query
    result = cursor.fetchall()
    for row in result:
        print(row)

In [ ]:
# Modificamos tipo de dato para un campo con ALTER

alter_table_query = """
    ALTER TABLE movies
        MODIFY COLUMN collection_in_mil DECIMAL(4,1)
"""
show_table_query = "DESCRIBE movies"
with connection.cursor() as cursor:
    cursor.execute(alter_table_query)
    cursor.execute(show_table_query)
    # Fetch rows from last executed query
    result = cursor.fetchall()
    print("Movie Table Schema after alteration:")
    for row in result:
        print(row)

In [ ]:
drop_table_query = "DROP TABLE ratings"
with connection.cursor() as cursor:
    cursor.execute(drop_table_query)

In [ ]:
show_table_query = "DESCRIBE ratings"
with connection.cursor() as cursor:
    try:
        cursor.execute(show_table_query)
        # Fetch rows from last executed query
        result = cursor.fetchall()
        for row in result:
            print(row)
    except Exception as e:
        print(str(e))

In [ ]:
# Creamos nuevamente la tabla ratings para el ejercicio

with connection.cursor() as cursor:
    cursor.execute(create_ratings_table_query)
    connection.commit()

## Insertando registros en una tabla

In [ ]:
# Insertando registros en movies usando execute()

insert_movies_query = """
    INSERT INTO movies (title, release_year, genre, collection_in_mil)
    VALUES
        ("Forrest Gump", 1994, "Drama", 330.2),
        ("3 Idiots", 2009, "Drama", 2.4),
        ("Eternal Sunshine of the Spotless Mind", 2004, "Drama", 34.5),
        ("Good Will Hunting", 1997, "Drama", 138.1),
        ("Skyfall", 2012, "Action", 304.6),
        ("Gladiator", 2000, "Action", 188.7),
        ("Black", 2005, "Drama", 3.0),
        ("Titanic", 1997, "Romance", 659.2),
        ("The Shawshank Redemption", 1994, "Drama",28.4),
        ("Udaan", 2010, "Drama", 1.5),
        ("Home Alone", 1990, "Comedy", 286.9),
        ("Casablanca", 1942, "Romance", 1.0),
        ("Avengers: Endgame", 2019, "Action", 858.8),
        ("Night of the Living Dead", 1968, "Horror", 2.5),
        ("The Godfather", 1972, "Crime", 135.6),
        ("Haider", 2014, "Action", 4.2),
        ("Inception", 2010, "Adventure", 293.7),
        ("Evil", 2003, "Horror", 1.3),
        ("Toy Story 4", 2019, "Animation", 434.9),
        ("Air Force One", 1997, "Drama", 138.1),
        ("The Dark Knight", 2008, "Action",535.4),
        ("Bhaag Milkha Bhaag", 2013, "Sport", 4.1),
        ("The Lion King", 1994, "Animation", 423.6),
        ("Pulp Fiction", 1994, "Crime", 108.8),
        ("Kai Po Che", 2013, "Sport", 6.0),
        ("Beasts of No Nation", 2015, "War", 1.4),
        ("Andadhun", 2018, "Thriller", 2.9),
        ("The Silence of the Lambs", 1991, "Crime", 68.2),
        ("Deadpool", 2016, "Action", 363.6),
        ("Drishyam", 2015, "Mystery", 3.0)
"""
with connection.cursor() as cursor:
    try:
        cursor.execute(insert_movies_query)
        connection.commit()
    except:
        connection.rollback()

In [ ]:
# Insertando registros en reviewers usando executemany()

insert_reviewers_query = """
    INSERT INTO reviewers (first_name, last_name)
        VALUES ( %s, %s )
"""
reviewers_records = [
    ("Chaitanya", "Baweja"),
    ("Mary", "Cooper"),
    ("John", "Wayne"),
    ("Thomas", "Stoneman"),
    ("Penny", "Hofstadter"),
    ("Mitchell", "Marsh"),
    ("Wyatt", "Skaggs"),
    ("Andre", "Veiga"),
    ("Sheldon", "Cooper"),
    ("Kimbra", "Masters"),
    ("Kat", "Dennings"),
    ("Bruce", "Wayne"),
    ("Domingo", "Cortes"),
    ("Rajesh", "Koothrappali"),
    ("Ben", "Glocker"),
    ("Mahinder", "Dhoni"),
    ("Akbar", "Khan"),
    ("Howard", "Wolowitz"),
    ("Pinkie", "Petit"),
    ("Gurkaran", "Singh"),
    ("Amy", "Farah Fowler"),
    ("Marlon", "Crafford"),
]
with connection.cursor() as cursor:
    try:
        cursor.executemany(insert_reviewers_query, reviewers_records)
        connection.commit()
    except:
        connection.rollback()

In [ ]:
# Insertando registros en ratings usando executemany()

insert_ratings_query = """
    INSERT INTO ratings (rating, movie_id, reviewer_id)
        VALUES ( %s, %s, %s)
"""
ratings_records = [
    (6.4, 17, 5), (5.6, 19, 1), (6.3, 22, 14), (5.1, 21, 17),
    (5.0, 5, 5), (6.5, 21, 5), (8.5, 30, 13), (9.7, 6, 4),
    (8.5, 24, 12), (9.9, 14, 9), (8.7, 26, 14), (9.9, 6, 10),
    (5.1, 30, 6), (5.4, 18, 16), (6.2, 6, 20), (7.3, 21, 19),
    (8.1, 17, 18), (5.0, 7, 2), (9.8, 23, 3), (8.0, 22, 9),
    (8.5, 11, 13), (5.0, 5, 11), (5.7, 8, 2), (7.6, 25, 19),
    (5.2, 18, 15), (9.7, 13, 3), (5.8, 18, 8), (5.8, 30, 15),
    (8.4, 21, 18), (6.2, 23, 16), (7.0, 10, 18), (9.5, 30, 20),
    (8.9, 3, 19), (6.4, 12, 2), (7.8, 12, 22), (9.9, 15, 13),
    (7.5, 20, 17), (9.0, 25, 6), (8.5, 23, 2), (5.3, 30, 17),
    (6.4, 5, 10), (8.1, 5, 21), (5.7, 22, 1), (6.3, 28, 4),
    (9.8, 13, 1)
]
with connection.cursor() as cursor:
    try:
        cursor.executemany(insert_ratings_query, ratings_records)
        connection.commit()
    except:
        connection.rollback()

## Consultando registros de la base de datos

In [ ]:
# Consulta de peliculas usando SELECT

select_movies_query = "SELECT * FROM movies LIMIT 5"
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    result = cursor.fetchall()
    for row in result:
        print(row)

In [ ]:
# Consulta de peliculas usando SELECT, agrega offset al limit

select_movies_query = "SELECT * FROM movies LIMIT 2,5"
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    result = cursor.fetchall()
    for row in result:
        print(row)

In [ ]:
# Consulta de peliculas usando SELECT, retorna columnas especificas

select_movies_query = "SELECT title, release_year FROM movies LIMIT 5"
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for row in cursor.fetchall():
        print(row)

**Ejercicio:** Realice una consulta de películas desde la posición 5 y extraiga las siguientes 10. Muestre únicamente el nombre y género de la película.

## Filtrando resultados usando ```where```

In [ ]:
# Consulta las peliculas con recaudación en taquilla > 300 millones

select_movies_query = """
    SELECT title, collection_in_mil
    FROM movies
    WHERE collection_in_mil > 300
    ORDER BY collection_in_mil DESC
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchall():
        print(movie)

In [ ]:
# Consulta peliculas y retorna concatenación de titulo y año

select_movies_query = """
    SELECT CONCAT(title, " (", release_year, ")"),
          collection_in_mil
    FROM movies
    ORDER BY collection_in_mil DESC
    LIMIT 5
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchall():
        print(movie)

In [ ]:
# Consulta peliculas y recupera únicamente 5 de ellas

select_movies_query = """
    SELECT CONCAT(title, " (", release_year, ")"),
          collection_in_mil
    FROM movies
    ORDER BY collection_in_mil DESC
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchmany(size=5):
        print(movie)
    cursor.fetchall()

**Ejercicio:** Consulte peliculas cuyo año de lanzamiento este por debajo de 2000 y ordene los resultados por recaudo en taquilla de mayor a menor valor.

## Manejando múltiples tablas usando ```join```

In [ ]:
# Consultar el nombre de las cinco películas mejor calificadas:

select_movies_query = """
    SELECT title, AVG(rating) as average_rating
    FROM ratings
    INNER JOIN movies
        ON movies.id = ratings.movie_id
    GROUP BY movie_id
    ORDER BY average_rating DESC
    LIMIT 5
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchall():
        print(movie)

In [ ]:
# Consultar el nombre del critico que otorgó la mayor cantidad de calificaciones

select_movies_query = """
    SELECT CONCAT(first_name, " ", last_name), COUNT(*) as num
    FROM reviewers
    INNER JOIN ratings
        ON reviewers.id = ratings.reviewer_id
    GROUP BY reviewer_id
    ORDER BY num DESC
    LIMIT 1
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchall():
        print(movie)

**Ejercicio.** Consultar el titulo, año de lanzamiento, cantidad de calificaciones y raiting de las peliculas top 10 en calificación.

## Actualización y eliminación de registros

### Comando ```update```

In [ ]:
# Actualización del apellido de un critico con nombre 'Amy'

update_query = """
    UPDATE reviewers
    SET last_name = "Cooper"
    WHERE first_name = "Amy"
"""
with connection.cursor() as cursor:
    try:
        cursor.execute(update_query)
        connection.commit()
    except Exception as e:
        connection.rollback()
        print(str(e))

In [ ]:
# Actualización de rating de una pelicula y un critico especificos

movie_id = input("Enter movie id: ")
reviewer_id = input("Enter reviewer id: ")
new_rating = input("Enter new rating: ")
update_query = """
    UPDATE ratings
    SET rating = "%s"
    WHERE movie_id = "%s" AND reviewer_id = "%s";

    SELECT *
    FROM ratings
    WHERE movie_id = "%s" AND reviewer_id = "%s"
""" % (
    new_rating,
    movie_id,
    reviewer_id,
    movie_id,
    reviewer_id,
)

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database="online_movie_rating",
    ) as connection:
        with connection.cursor() as cursor:
            try:
                for result in cursor.execute(update_query, multi=True):
                    if result.with_rows:
                        print(result.fetchall())
                connection.commit()
            except Exception as e:
                connection.rollback()
                print(str(e))
except Error as e:
    print(e)

In [ ]:
# Actualización de rating de una pelicula y un critico especificos pasando los argumentos

movie_id = input("Enter movie id: ")
reviewer_id = input("Enter reviewer id: ")
new_rating = input("Enter new rating: ")
update_query = """
    UPDATE ratings
    SET rating = %s
    WHERE movie_id = %s AND reviewer_id = %s;

    SELECT *
    FROM ratings
    WHERE movie_id = %s AND reviewer_id = %s
"""
val_tuple = (
    new_rating,
    movie_id,
    reviewer_id,
    movie_id,
    reviewer_id,
)

try:
    with connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database="online_movie_rating",
    ) as connection:
        with connection.cursor() as cursor:
            try:
                for result in cursor.execute(update_query, val_tuple, multi=True):
                    if result.with_rows:
                        print(result.fetchall())
                connection.commit()
            except Exception as e:
                connection.rollback()
                print(str(e))
except Error as e:
    print(e)

### Comando ```delete```

In [ ]:
select_movies_query = """
    SELECT reviewer_id, movie_id FROM ratings
    WHERE reviewer_id = 2
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchall():
        print(movie)

In [ ]:
delete_query = "DELETE FROM ratings WHERE reviewer_id = 2"
with connection.cursor() as cursor:
    try:
        cursor.execute(delete_query)
        connection.commit()
    except Exception as e:
        connection.rollback()
        print(str(e))

In [ ]:
select_movies_query = """
    SELECT reviewer_id, movie_id FROM ratings
    WHERE reviewer_id = 2
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchall():
        print(movie)

## Otras formas de conectarse con MySQL

**Mapeo objeto-relacional (ORM)** es una técnica que le permite consultar y manipular datos de una base de datos directamente utilizando un lenguaje orientado a objetos. Una biblioteca ORM encapsula el código necesario para manipular los datos, lo que elimina la necesidad de usar incluso un poquito de SQL. Estos son los ORM de Python más populares para bases de datos basadas en SQL:

- **[SQLAlchemy](https://docs.sqlalchemy.org/en/13/index.html)** es un ORM que facilita la comunicación entre Python y otras bases de datos SQL. Puede conectarse a diferentes motores de bases de datos como MySQL, PostgreSQL, SQLite, etc. SQLAlchemy se usa comúnmente junto con *pandas* para proporcionar una funcionalidad completa de manejo de datos.

- **[peewee](https://docs.peewee-orm.com/en/latest/)** es un ORM ligero y rápido que se configura rápidamente. Esto es bastante útil cuando su interacción con la base de datos se limita a extraer unos pocos registros. Por ejemplo, si necesita copiar registros seleccionados de una base de datos MySQL en un archivo CSV, entonces peewee podría ser su mejor opción.

- **[Django ORM](https://books.agiliq.com/projects/django-orm-cookbook/en/latest/introduction.html)** es una de las características más potentes de Django. Puede interactuar con una gran variedad de bases de datos como SQLite, PostgreSQL y MySQL. Muchas aplicaciones basadas en Django usan Django ORM para el modelado de datos y consultas básicas, pero a menudo cambian a SQLAlchemy para requisitos más complejos.